# Dump salaries from 990EZ in OR (continuing from part 1)

In [4]:
import unicodecsv as csv
from irsx.xmlrunner import XMLRunner
import pandas as pd

In [5]:
# read the whole file of orgs with efilings from part 1 here, it's not very long
file_rows = [] 
# We're using the output of part 1
with open('orefilers.csv', 'rb') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        file_rows.append(row)
        

In [6]:
# the name of the output file
outfilename ="ez_comp_1.csv"
outfile = open(outfilename , 'wb')

# the header rows as they'll appear in the output
headers = ["period", "ein", "object_id", "taxpayer_name", "name", "business_name1", "business_name2", "title", "org_comp", "related_comp", "other_comp", "gross_receipts"]
# start up a dictwriter, ignore extra rows
dw = csv.DictWriter(outfile, headers, extrasaction='ignore')
dw.writeheader()

In [7]:
# get an XMLRunner -- this is what actually does the parsing
xml_runner = XMLRunner()

## Figure out what to extract

Data from each repeating group should go to it's own file, otherwise it won't make sense.

To figure out what to capture, I started by looking at schedule J: http://www.irsx.info/#IRS990ScheduleJ
Then I went to the table details and picked the rows I wanted from the repeating group:
http://www.irsx.info/metadata/groups/SkdJRltdOrgOffcrTrstKyEmpl.html

Note that it's common for director/employee names in schedule J to get listed as businessname.

Also note that IRSx checks to see if a file has been downloaded before fetching it. Running this the first time will be slow if the filings aren't already downloaded, but much faster if they've already been downloaded.

-------NOTE--------
Edited to get compensation from EZ partIV, not sked j


In [8]:
DEMO_MAX = 10
num_rows = 0

for row in file_rows:
    num_rows += 1
    this_object_id = row['OBJECT_ID']
    parsed_filing = xml_runner.run_filing(this_object_id)
    
    # if it somehow busted, just note it and continue
    if not parsed_filing:
        print("Skipping filing %s(filings with pre-2013 schemas are skipped)\n row details: %s" % (this_object_id, row))
        continue 
    
    schedule_list = parsed_filing.list_schedules()
    if 'IRS990EZ' in schedule_list:
        
        # store the output in this dict
        outputdata = {}
        # assign some initial values from the input csv
        outputdata['period'] = row['TAX_PERIOD_x']
        outputdata['ein'] = row['EIN']
        outputdata['object_id'] = row['OBJECT_ID']
        outputdata['taxpayer_name'] = row['TAXPAYER_NAME']
        
        # some schedules can appear multiple times, but irs990 only appears once
        # so we grab the first one 
        parsed_main = parsed_filing.get_parsed_sked('IRS990EZ')[0] 
        
        
        print(parsed_main)
        
        
        
        
#         # now use the table name we looked up -- that's where we find it
#         try:
#             # repeating groups are returned as an array of dicts
#             employee_groups = parsed_main['groups']['Frm990PrtVIISctnA']
#         except KeyError:
#             print("No Frm990PrtVIISctnA found in %s skipping" % this_object_id)
#             continue
          
#         # read through each employee and pull out the data we want
#         for employee_group in employee_groups:
            
#             # That leaves the following values to come from schedule J if there is one
#             # "name", "business_name1", "business_name2", "title", "org_comp", "related_comp"
#             # those keys come from the headers we gave dictwriter before
#             outputdata['name'] = employee_group.get('PrsnNm')
#             outputdata['business_name1'] = employee_group.get('BsnssNmLn1Txt')
#             outputdata['business_name2'] = employee_group.get('BsnssNmLn2Txt')
#             outputdata['title'] = employee_group.get('TtlTxt')
#             outputdata['org_comp'] = employee_group.get('RprtblCmpFrmOrgAmt')  
#             outputdata['related_comp'] = employee_group.get('RprtblCmpFrmRltdOrgAmt')
#             outputdata['other_comp'] = employee_group.get('OthrCmpnstnAmt')
        
#             dw.writerow(outputdata)
        
#     else:
#         print("No 990 in filing %s, skipping" % this_object_id)

    # Don't run endlessly during a demo:
    if(num_rows > DEMO_MAX):
       break
    if num_rows%100==0:
        print("Processed %s filings" % num_rows)

Filing version 2017v2.3 isn't supported for this operation
Filing version 2017v2.3 isn't supported for this operation
Filing version 2017v2.3 isn't supported for this operation
Filing version 2017v2.3 isn't supported for this operation
Filing version 2017v2.3 isn't supported for this operation


In [9]:
# close the outfile
outfile.close()

sked_j_ore_efilers = pd.read_csv(outfilename)
sked_j_ore_efilers.head()

,period,ein,object_id,taxpayer_name,name,business_name1,business_name2,title,org_comp,related_comp,other_comp,gross_receipts
